In [ ]:
import pandas as pd
import textwrap
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langfuse.langchain import CallbackHandler

In [ ]:
def ask_llm(prompt: str, model: str = "gpt-4.1", timeout: int = 60) -> str:
    """
    Send prompt to internal LLM API gateway. Returns the response text.
    Uses TokenFetcher for auth and Langfuse for observability.
    """
    try:
        # 1. Setup Environment Credentials
        os.environ['LLM_CLIENT_ID'] = os.getenv("LLM_CLIENT_ID")
        os.environ['LLM_CLIENT_SECRET'] = os.getenv("LLM_CLIENT_SECRET")

        # 2. Specific Langfuse Config
        LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
        LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
        LANGFUSE_HOST = os.getenv("LANGFUSE_HOST")

        # 3. Initialize Authentication
        token_fetcher = TokenFetcher(
            client_id=os.getenv('LLM_CLIENT_ID'),
            client_secret=os.getenv('LLM_CLIENT_SECRET'),
            auth_url="https://auth.llm.ai/oauth2/token",
            scope="llm.openai.api"
        )

        # 4. Initialize Observability (Langfuse)
        # The import path was fixed to use langfuse.langchain
        langfuse_handler = CallbackHandler(
            public_key=LANGFUSE_PUBLIC_KEY,
            secret_key=LANGFUSE_SECRET_KEY,
            host=LANGFUSE_HOST
        )

        # 5. Initialize the LLM Client
        llm = ChatOpenAI(
            model=model,
            # 'base_url' is preferred over 'openai_api_base' in newer langchain versions
            api_key=token_fetcher.token, 
            temperature=0,
            callbacks=[langfuse_handler],
            request_timeout=timeout
        )

        # 6. Invoke the model
        messages = [HumanMessage(content=prompt)]
        response = llm.invoke(messages)

        return response.content

    except ImportError as e:
        return f"[LLM-error] Missing dependency: {e}. Try: pip install langfuse langchain-openai"
    except Exception as e:
        return f"[LLM-failed] {str(e)}"

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
df = pd.read_csv('/Users/ms128/Library/CloudStorage/OneDrive-azureford/Desktop/AI-Assistant/Australia_service.csv')
print(df.select_dtypes(include='number').columns)

Index(['Month', 'Year', 'Retail_RO_Obj', 'Retail_RO_Act', 'Rev/RO_Obj',
       'Retail_Traffic_FL', 'Rev/RO Act', 'Battery_RO', 'Brake_RO', 'Wiper_RO',
       'CVP_Score', 'CVP_Survey Returns', 'NPS_Score', 'NPS_Survey Returns',
       'FIRFT_Score', 'FIRFT_Survey Returns', 'CVP_Score_3M',
       'CVP_Survey Returns_3M', 'NPS_Score_3M', 'NPS_Survey Returns_3M',
       'FIRFT_Score_3M', 'FIRFT_Survey Returns_3M', 'Total CRC Case',
       'CRC Dealer', 'CRC Reactive', 'CRC Proactive'],
      dtype='object')


In [10]:
def prompt_to_code(prompt: str, df: pd.DataFrame):
    """
    Convert known prompt templates into runnable python code strings.
    If the prompt is custom/unrecognized, return None (so UI can send to LLM instead).
    """
    p = prompt.strip().lower()

    # Summary
    if p.startswith("summarize the dataset"):
        code = textwrap.dedent("""
            # produce a short summary as printed text
            info = []
            info.append(f"Rows: {len(df)}, Columns: {len(df.columns)}")
            info.append("Column types: " + ", ".join([f\"{c}:{str(df[c].dtype)[:10]}\" for c in df.columns[:10]]))
            miss = df.isnull().sum().sort_values(ascending=False).head(10)
            info.append("Top missing: " + ", ".join([f\"{idx}:{val}\" for idx,val in miss.items() if val>0]))
            numeric = df.select_dtypes(include=['number']).columns.tolist()
            info.append(f\"Numeric columns count: {len(numeric)}\")
            # print concise bullets
            result = \"\\n\".join([\"- \"+i for i in info])
        """)
        return code

    # Top counts for categorical
    if "top 10 counts for the categorical column" in p or "top 10 counts" in p and "'" in p:
        # try to extract column name between quotes
        import re
        m = re.search(r"'([^']+)'", prompt)
        if not m:
            m = re.search(r'"([^"]+)"', prompt)
        col = m.group(1) if m else None
        if col:
            code = textwrap.dedent(f"""
                # top 10 counts for '{col}'
                result = df['{col}'].value_counts(dropna=False).head(10).reset_index()
                result.columns = ['value','count']
            """)
            return code

    # Summary statistics for numeric
    if "summary statistics" in p or "describe" in p:
        code = textwrap.dedent("""
            result = df.select_dtypes(include=['number']).describe().T
        """)
        return code

    # Histogram
    if p.startswith("create a histogram of the numeric column") or "histogram of the numeric column" in p:
        import re
        m = re.search(r"'([^']+)'", prompt)
        col = m.group(1) if m else None
        if col:
            code = textwrap.dedent(f"""
                # histogram for '{col}'
                plt.figure(figsize=(6,4))
                df['{col}'].dropna().astype(float).hist(bins=30)
                plt.title('Histogram of {col}')
                plt.xlabel('{col}')
                plt.ylabel('count')
                # produce an image by saving to result_img_path variable
                result_img_path = None
            """)
            # We'll return plotting code that uses plt; execution will save figure
            return code

    # Scatter plot
    if "scatter plot comparing" in p and "vs" in p:
        import re
        m = re.search(r"'([^']+)' \\(x\\) vs '([^']+)' \\(y\\)", prompt)
        if m:
            xcol, ycol = m.group(1), m.group(2)
            code = textwrap.dedent(f"""
                plt.figure(figsize=(6,4))
                df.plot.scatter(x='{xcol}', y='{ycol}')
                plt.title('{ycol} vs {xcol}')
                result_img_path = None
            """)
            return code

    # Top N rows sorted by col
    if p.startswith("show the top 10 rows sorted by"):
        import re
        m = re.search(r"by '([^']+)'", prompt)
        if m:
            col = m.group(1)
            code = textwrap.dedent(f"""
                result = df.sort_values('{col}', ascending=False).head(10).reset_index(drop=True)
            """)
            return code

    # Time series monthly sum
    # if "monthly sum" in p and "using the datetime column" in p:
    #     import re
    #     m = re.search(r"sum of '([^']+)' using the datetime column '([^']+)'", prompt)
    #     if m:
    #         ag, dcol = m.group(1), m.group(2)
    #         code = textwrap.dedent(f"""
    #             tmp = df.copy()
    #             tmp['{dcol}'] = pd.to_datetime(tmp['{dcol}'], errors='coerce')
    #             res = tmp.dropna(subset=['{dcol}'])
    #             res = res.set_index('{dcol}').resample('M')['{ag}'].sum().reset_index()
    #             result = res
    #         """)
    #         return code

    # Counts per month (datetime only)
    # if "counts per month using the datetime column" in p:
    #     import re
    #     m = re.search(r"datetime column '([^']+)'", prompt)
    #     dcol = m.group(1) if m else None
    #     if dcol:
    #         code = textwrap.dedent(f"""
    #             tmp = df.copy()
    #             tmp['{dcol}'] = pd.to_datetime(tmp['{dcol}'], errors='coerce')
    #             res = tmp.dropna(subset=['{dcol}']).set_index('{dcol}').resample('M').size().reset_index(name='count')
    #             result = res
    #         """)
    #         return code

    # Correlation heatmap
    if "correlation matrix heatmap" in p or "correlation heatmap" in p:
        code = textwrap.dedent("""
            corr = df.select_dtypes(include=['number']).corr()
            import matplotlib.pyplot as plt
            plt.figure(figsize=(6,5))
            plt.imshow(corr, cmap='viridis', aspect='auto')
            plt.colorbar()
            plt.xticks(range(len(corr)), corr.columns, rotation=90)
            plt.yticks(range(len(corr)), corr.columns)
            plt.title('Correlation matrix')
            result_img_path = None
        """)
        return code

    # Anomaly detection using z-score
    if "anomalies" in p and "z-score" in p:
      code = textwrap.dedent("""
          from scipy import stats
          import numpy as np
          import pandas as pd
          
          # Select numeric columns only
          operational_cols = [
          "Retail_RO_Obj",
          "Retail_RO_Act",
          "Retail_RO_Parts_Revenue",
          "Retail_Traffic_FL",
          "Battery_RO",
          "Brake_RO",
          "Wiper_RO",
          "Base_VIN",
          "Retained_VIN",
          "Non Retailed_VIN",
          "Total CRC Case",
          "CRC Dealer",
          "CRC Reactive",
          "CRC Proactive"
          ]

          num = df[operational_cols].select_dtypes(include=['number'])
          
          # Compute z-score column-wise, ignoring NaNs
          z = np.abs(stats.zscore(num, nan_policy='omit'))
          
          # zscore returns a numpy array; convert to DataFrame and align index
          z_df = pd.DataFrame(z, columns=num.columns, index=df.index)
          
          # Mask: any z-score > 3 in any numeric column
          mask = (z_df > 3).any(axis=1)
          
          # Show top 20 anomalies
          result = df.loc[mask].head(20).reset_index(drop=True)
      """)
    return code

    # Unknown / custom prompts -> return None
    return None

In [5]:
def ask_llm(prompt: str, model: str = "gpt-5-mini-2025-08-07", timeout: int = 60) -> str:
    """
    Send prompt to LLM API gateway. Returns the response text.
    Uses TokenFetcher for auth and Langfuse for observability.
    """
    # Load environment variables from .env file
    load_dotenv()
    
    try:
        # 1. Setup Environment Credentials
        os.environ['CLIENT_ID'] = os.getenv("CLIENT_ID")
        os.environ['CLIENT_SECRET'] = os.getenv("CLIENT_SECRET")

        # 2. Initialize Authentication
        token_fetcher = TokenFetcher() 

        # 3. Initialize Observability (Langfuse)
        # The import path was fixed to use langfuse.langchain
        # use the CallbackHandler class that we imported above
        langfuse_handler = CallbackHandler()

        # Get BASE_URL from environment variable
        url = os.getenv("BASE_URL")
        if not url:
            raise ValueError("BASE_URL is not set")

        # 4. Initialize the LLM Client
        llm = ChatOpenAI(
            model = model,
            base_url = url,
            api_key = token_fetcher.token, 
            temperature = 0,
            callbacks = [langfuse_handler],
            request_timeout = timeout
        )

        # 5. Invoke the model
        messages = [HumanMessage(content=prompt)]
        response = llm.invoke(messages)

        return response.content

    except ImportError as e:
        return f"[LLM-error] Missing dependency: {e}. Try: pip install langfuse langchain-openai"
    except Exception as e:
        return f"[LLM-failed] {str(e)}"